In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
from src.tools import visualize_tool, safe_dataframe_tool
import openai
from IPython.display import HTML
from src.classifiers import is_uae_real_estate_query, llm_classifier


load_dotenv()

True

In [2]:
official_ai = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def main_agent(query: str):
    if not is_uae_real_estate_query(query):
        return {"type": "output", "data": "This is an irrelevant question to UAE property."}

    data = safe_dataframe_tool(query)
    if isinstance(data, dict) and data.get("error"):
        return data

    action = llm_classifier(query)

    if action == "output":
        return {"type": "data", "data": data}

    elif action == "plot_stats":
        result = visualize_tool.invoke({"data": data, "query": query})
        return {"type": "plot", **result}

    elif action == "geospatial_plot":
        return {"error": "Geospatial plotting tool not implemented yet."}

    else:
        return {"error": f"Unknown action '{action}' from classifier."}



In [6]:
result = main_agent("show 5 most expensive properties in Duabai")

/tmp/ipykernel_100788/3144146417.py:5: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  data = safe_dataframe_tool(query)


                                                  title  \
3612          Ultra-Luxury | Duplex Penthouse | Branded   
1594  Ultra-Luxury Shell and Core Mansion |On Lagoon...   
1727           Golf Facing Mansion | Vacant on Transfer   
2987        Freehold Independent  Brand-New Ready Villa   
427     Luxurious &amp; Elegant | Ready 2025 | Sea View   

                                         displayAddress  bathrooms  bedrooms  \
3612     Binghatti Mercedes Benz, Downtown Dubai, Dubai       <NA>         5   
1594  District One Villas, District One, Mohammed Bi...       <NA>      <NA>   
1727        Dubai Hills View, Dubai Hills Estate, Dubai       <NA>      <NA>   
2987       Al Wasl Villas, Al Wasl Road, Al Wasl, Dubai       <NA>      <NA>   
427   Bulgari Resort & Residences, Jumeirah Bay Isla...       <NA>      <NA>   

                       addedOn                  type      price  verified  \
3612 2024-08-16 09:13:16+00:00  Residential for Sale  199000000     False   
1594 2024-

In [8]:
result["data"]

[{'title': 'Ultra-Luxury | Duplex Penthouse | Branded',
  'displayAddress': 'Binghatti Mercedes Benz, Downtown Dubai, Dubai',
  'bathrooms': None,
  'bedrooms': 5,
  'addedOn': Timestamp('2024-08-16 09:13:16+0000', tz='UTC'),
  'type': 'Residential for Sale',
  'price': 199000000,
  'verified': False,
  'priceDuration': 'sell',
  'sizeMin': nan,
  'furnishing': 'YES',
  'description': '<p>The Mercedes-Benz Vision EQ Silver Arrow Penthouse!</p><p><br></p><p>Named after the groundbreaking Mercedes-Benz Vision</p><p>EQ Silver Arrow, this Duplex Penthouse echoes the same</p><p>exalting moments of this legend.</p><p><br></p><p>Experience unparalleled luxury at Mercedes-Benz by Binghatti with this extraordinary 5-bedroom penthouse Duplex. This residence offers a seamless fusion of contemporary design and ultimate comfort comprising 2 floors. </p><p><br></p><p>Enjoy top-tier amenities, including a state-of-the-art gym, a refreshing pool, and a rejuvenating spa, all while benefiting from round

In [1]:
# from IPython.display import HTML


# html_img = f'<img src="data:image/png;base64,{result["image"]}" />'
# display(HTML(html_img))